In [298]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from timeit import Timer
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.nn.init import kaiming_normal_

In [299]:
torch.cuda.get_device_name(0), torch.cuda.current_device()

('GeForce MX150', 0)

In [168]:
# class Data(Dataset):
#     def __init__(self, transforms, train = False):
#         self.x = torch.zeros(1)
#         self.y = torch.zeros(1)
        
#         negative_file_path = os.path.join('concrete_class/', 'Negative')
#         negative_files = [file for file in os.listdir(negative_file_path) if file.endswith('.jpg')]
#         negative_files.sort()
#         positive_file_path = os.path.join('concrete_class/', 'Positive')
#         positive_files = [file for file in os.listdir(positive_file_path) if file.endswith('.jpg')]
#         positive_files.sort()
#         print(negative_files[0], type(negative_files[0]))
        
#         if train:
#             self.x = torch.zeros(36000, 1)
#             for i in range(36000):
#                 self.x[i] = transforms(negative_files[i]) if i < 18000 else transforms(negative_files[i])
#             self.y = torch.zeros(36000, 1, dtype = 'uint8')
#             self.y[18000:, :] = 1
#         else:
#             self.x = torch.zeros(4000, 1)
#             for i in range(4000):
#                 self.x[i] = transforms(negative_files[i + 36000]) if i < 2000 else transforms(negative_files[i + 38000])
#             self.y = torch.zeros(4000, 1, dtype = 'uint8')
#             self.y[2000:, :] = 1
        
#     def __getitem__(self, index):
#         return self.y[index], self.x[index]
#     def __len__(self):
#         return self.y.shape[0]

In [325]:
class CNN(nn.Module):
    def __init__(self, out_1 = 16, out_2 = 32, out_3 = 64):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = out_1, kernel_size = 5)
        # self.drop2d1 = nn.Dropout2d(p = 0.35, inplace = True)
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 1)
        self.batch_norm1 = nn.BatchNorm2d(out_1, momentum = 0.5)

        self.conv2 = nn.Conv2d(in_channels = out_1, out_channels = out_2, kernel_size = 5)
        # self.drop2d2 = nn.Dropout2d(p = 0.35, inplace = True)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 1)
        self.batch_norm2 = nn.BatchNorm2d(out_2, momentum = 0.5)

        self.conv3 = nn.Conv2d(in_channels = out_2, out_channels = out_3, kernel_size = 5)
        # self.drop2d3 = nn.Dropout2d(p = 0.35, inplace = True)
        self.maxpool3 = nn.MaxPool2d(kernel_size = 2, stride = 1)
        self.batch_norm3 = nn.BatchNorm2d(out_3, momentum = 0.5)
        
        self.flatten = nn.Flatten()

        self.linear1 = nn.Linear(2876416, 200)
        self.drop1 = nn.Dropout(p = 0.35, inplace = True)
        self.linear2 = nn.Linear(200, 100)
        self.drop2 = nn.Dropout(p = 0.35, inplace = True)
        self.linear_output = nn.Linear(100, 10)

        kaiming_normal_(self.conv1.weight, nonlinearity = 'relu')
        kaiming_normal_(self.conv2.weight, nonlinearity = 'relu')
        kaiming_normal_(self.linear1.weight, nonlinearity = 'relu')
        kaiming_normal_(self.linear2.weight, nonlinearity = 'relu')
        kaiming_normal_(self.linear_output.weight, nonlinearity = 'relu')

    def forward(self, x):
        x = self.conv1(x)
        # x = self.drop2d1(x)
        x = self.maxpool1(x)
        x = self.batch_norm1(x)
        x = torch.relu(x)
        
        x = self.conv2(x)
        # x = self.drop2d2(x)
        x = self.maxpool2(x)
        x = self.batch_norm2(x)
        x = torch.relu(x)
        
        x = self.conv3(x)
        # x = self.drop2d3(x)
        x = self.maxpool3(x)
        x = self.batch_norm3(x)
        x = torch.relu(x)
        
        x = self.flatten(x)
        
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.drop1(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.drop2(x)
        x = self.linear_output(x)
        return x

In [326]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs):
    loss_acc = {'training_loss': [], 'training_acc': [], 'validation_loss': [], 'validation_acc': []}
    for epoch in range(epochs):
        tr_time = Timer()
        tr_acc = 0
        val_acc = 0
        
        for i, (x, y) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            z = model.forward(x)
            tr_loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            loss_acc.training_loss.append(tr_loss)
            
        tr_acc = 100 * (tr_acc / len(train_loader))
        loss_acc.training_acc.append(tr_acc)
        
        val_time = Timer()
        for x, y in validation_loader:
            model.eval()
            z = model(x)
            val_loss = criterion(z, y)
            _, y_hat = torch.max(z, 1)
            val_acc += (y_hat == y).sum().item()
            loss_acc.validation_loss.append(val_loss)
            
        val_acc = 100 * (val_acc / len(train_loader))
        loss_acc.validation_acc.append(val_acc)
        print('Epoch: {}, training accuracy/time: {}, {}, validation accuracy/time: {}, {}'
              .format(epoch, tr_acc, tr_time.timeit(), val_acc, val_timer.timeit()))

In [327]:
class ToCudaTensor(transforms.ToTensor):
    def __call__(self, image):
        img = np.asarray(image)
        img = img.reshape((3, 227, 227))
        return torch.from_numpy(img).float().to(0)
    def __repr__(self):
        return self.__class__.__name__ + '()'

In [328]:
# transformer = transforms.ToTensor()
# train_data = Data(transforms = transformer, train = True)
# val_data = Data(transforms = transformer, train = False)
images = datasets.ImageFolder(root = 'concrete_class/', transform = ToCudaTensor())

train_loader = DataLoader(images, shuffle = True)
val_loader = DataLoader(images, shuffle = True)

In [329]:
np.shape(images[0][0]), images[0]

(torch.Size([3, 227, 227]),
 (tensor([[[178., 175., 170.,  ..., 170., 180., 177.],
           [172., 181., 178.,  ..., 179., 174., 180.],
           [179., 174., 171.,  ..., 189., 186., 179.],
           ...,
           [173., 168., 173.,  ..., 182., 177., 174.],
           [162., 158., 155.,  ..., 152., 163., 159.],
           [156., 167., 163.,  ..., 176., 171., 178.]],
  
          [[175., 170., 176.,  ..., 181., 176., 173.],
           [164., 160., 157.,  ..., 152., 163., 159.],
           [156., 167., 163.,  ..., 178., 173., 182.],
           ...,
           [171., 177., 176.,  ..., 174., 171., 176.],
           [172., 169., 176.,  ..., 179., 175., 172.],
           [151., 150., 146.,  ..., 170., 178., 177.]],
  
          [[173., 180., 179.,  ..., 172., 169., 174.],
           [170., 167., 176.,  ..., 177., 173., 170.],
           [161., 160., 156.,  ..., 168., 177., 176.],
           ...,
           [168., 164., 161.,  ..., 171., 177., 174.],
           [169., 173., 170.,  ..., 

In [330]:
images.classes

['Negative', 'Positive']

In [331]:
model = CNN()
model.cuda(0)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 5

RuntimeError: CUDA out of memory. Tried to allocate 2.14 GiB (GPU 0; 2.00 GiB total capacity; 322.63 MiB already allocated; 936.88 MiB free; 376.00 MiB reserved in total by PyTorch)

In [ ]:
model.modules

In [ ]:
train(model, criterion, optimizer, train_loader, val_loader, epochs)